In [126]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [76]:
train_df = pd.read_csv('train.csv')
print train_df.columns

Index([u'id', u'qid1', u'qid2', u'question1', u'question2', u'is_duplicate'], dtype='object')


In [77]:
train_q1 = train_df['question1'].tolist()
train_q2 = train_df['question2'].tolist()
train_y = train_df['is_duplicate'].tolist()

In [78]:
print len(train_q1)
print len(train_q2)

del_idxs = []

for i,q in enumerate(train_q1):
    if type(q) != str:
        del_idxs.append(i)
print del_idxs
for i,q in enumerate(train_q2):
    if type(q) != str:
        del_idxs.append(i)
print del_idxs
del_idxs = list(set(del_idxs))

train_q1 = [train_q1[i] for i in range(len(train_q1)) if i not in del_idxs]    
train_q2 = [train_q2[i] for i in range(len(train_q2)) if i not in del_idxs]    
train_y = [train_y[i] for i in range(len(train_y)) if i not in del_idxs]    
print len(train_q1)
print len(train_q2)
print len(train_y)

404290
404290
[363362]
[363362, 105780, 201841]
404287
404287
404287


In [79]:
#nltk.download('stopwords')
tbl = dict.fromkeys(i for i in xrange(sys.maxunicode) if unicodedata.category(unichr(i)).startswith('P'))
stop_words = set(stopwords.words('english'))
def tokenize(text):
    #print text
    text = unicode(text)
    #text = re.sub(r'\d+', '', text)
    #print text
    tokens = nltk.word_tokenize(text)
    
    stems = []
    for item in tokens:
       if item not in stop_words:
            #stems.append(item)
           stems.append(PorterStemmer().stem(item))
    return stems

def preprocess(text):
    if type(text)==float:
        print text
    text = text.lower().translate(None, string.punctuation)
    return text

In [80]:
# Remove punctuation

train_q1 = [preprocess(q) for q in train_q1]
train_q2 = [preprocess(q) for q in train_q2]

In [81]:
# stemming and tokenize

wordCount = defaultdict(int)
for i,q in enumerate(train_q1):
    words = tokenize(q)
    for w in words:
        wordCount[w]+=1

In [82]:
for i,q in enumerate(train_q2):
    words = tokenize(q)
    for w in words:
        wordCount[w]+=1
        
print len(wordCount.keys())

86598


In [83]:
# Limit the vocabulary
uni_sorted = sorted(wordCount.items(), key=operator.itemgetter(1)) [::-1]
vocab = [item[0] for item in uni_sorted[:10000]]
print vocab[:100]

[u'best', u'get', u'india', u'like', u'use', u'peopl', u'good', u'way', u'differ', u'would', u'make', u'one', u'quora', u'learn', u'time', u'life', u'work', u'know', u'money', u'year', u'thing', u'think', u'mean', u'question', u'what', u'go', u'indian', u'new', u'much', u'start', u'someon', u'engin', u'book', u'mani', u'becom', u'find', u'want', u'job', u'day', u'movi', u'trump', u'us', u'happen', u'ever', u'without', u'world', u'feel', u'onlin', u'better', u'person', u'girl', u'take', u'first', u'note', u'love', u'compani', u'need', u'dont', u'univers', u'account', u'english', u'possibl', u'improv', u'phone', u'buy', u'could', u'program', u'prepar', u'2016', u'number', u'countri', u'compar', u'ask', u'chang', u'live', u'languag', u'weight', u'student', u'realli', u'studi', u'app', u'googl', u'donald', u'lose', u'stop', u'busi', u'500', u'1000', u'answer', u'friend', u'develop', u'im', u'websit', u'import', u'long', u'war', u'effect', u'look', u'colleg', u'place']


In [84]:
# Create a single list of questions
train = train_q1+train_q2
vectorizer = TfidfVectorizer(tokenizer=tokenize, vocabulary=vocab)
train_tfidf = vectorizer.fit_transform(train)

In [85]:
print train_tfidf.shape
print len(train_q1)
print len(train_q2)
print len(train)

(808574, 10000)
404287
404287
808574


In [86]:
def csr_cosine_dist(A,B):
    A_d = sparse.csr_matrix.todense(A)
    B_d = sparse.csr_matrix.todense(B)
    return cosine(A_d,B_d)

distances = []
for i in range(len(train_q2)):
    j = i+len(train_q1)
    distances.append(csr_cosine_dist(train_tfidf[i], train_tfidf[j]))

In [88]:
print len(distances)
distances = np.array(distances)
train_y = np.array(train_y)

404287


In [123]:
thresh = 0.45
preds = distances<thresh
acc = np.mean(preds == train_y)
print acc
average_precision = average_precision_score(train_y, preds)

print('Average precision-recall score: {0:0.2f}'.format(
      average_precision))


0.629340048035
Average precision-recall score: 0.47


In [ ]:
precision, recall, _ = precision_recall_curve(train_y, preds)

plt.step(recall, precision, color='b', alpha=0.2,
         where='post')
plt.fill_between(recall, precision, step='post', alpha=0.2,
                 color='b')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.ylim([0.0, 1.05])
plt.xlim([0.0, 1.0])
plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
          average_precision))
plt.show()